solving Newsvendor problem modelled as 2 stage stochastic program

let D be RV of demand --> Normal

our decision is to decide the number of products to be kept in inventory at the begining of each day
cost of overstocking is $c_o$ and cost of understocking is $c_u$

min $\displaystyle\sum_w c_oo_w + c_uu_w$

$ i - o_w + u_w = d_w  \quad \forall w \in \Omega$ 

In [1]:
import numpy as np
from pyomo.environ import *
import scipy.stats as st

import plotly.graph_objects as go

In [2]:
mu = 50
sig = 10

demands = np.random.normal(mu,sig,9000)
c_o = 120
c_u = 250

In [3]:
### scenarios 

fig = go.Figure()

fig.add_trace(go.Histogram(x=demands))

fig.show()

In [4]:
model = ConcreteModel()

model.inv = Var(domain=NonNegativeReals)

model.o = Var(range(len(demands)),domain=NonNegativeReals)
model.u = Var(range(len(demands)),domain=NonNegativeReals)

model.constraints = ConstraintList()

for i,d in enumerate(demands):
    model.constraints.add(expr = model.inv - model.o[i] + model.u[i] == d)

model.obj = Objective(expr = sum([c_o*model.o[i]+c_u*model.u[i] for i in range(len(demands))]))

# model.pprint()

In [5]:
minotaur = SolverFactory("mbnb", executable='../CHL_Grouping/mbin/mbnb')
res = minotaur.solve(model, tee=False)
print(res.write())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 18001
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: 
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 1.3862967491149902
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
None


In [6]:
model.inv()

54.74585729260035

In [7]:
st.norm.ppf(c_u/(c_u+c_o),loc = mu,scale = sig)

54.556403137378794

Dual of NewsVendor (2SSP)

<center>

$\displaystyle\max_{\pi_\omega} \quad \pi_\omega^T (ex-d_\omega) \\$
$ s.t. \hspace{150pt} \\$
$c_u \geq \pi_\omega \geq -c_o$

</center>

In [8]:
def getDual(x,demand,co,cu):
    dual = []
    for d in demand:
        if d-x > 0:
            dual.append(cu)
        elif d-x < 0:
            dual.append(-co)
        else:
            dual.append(0)

    return np.array(dual) 

In [9]:
def getValue(dual,x,demand):
    return np.matmul(dual.T,(np.array(demand))-np.ones(len(demands))*x)

In [10]:
def getCost(x,demand,co,cu):
    sum = 0
    for d in demand:
        sum += max(0,x-d)*co + max(0,d-x)*cu
    
    return sum

In [11]:
vals = []
cost = []
m = []
c = []

rng = (40,70)
for x in range(rng[0],rng[1]):
    dual = getDual(x,demands,c_o,c_u)
    vals.append(getValue(dual,x,demands))
    cost.append(getCost(x,demands,c_o,c_u))
    m.append(sum(dual))
    c.append(np.matmul(dual.T,np.array(demands)))

fig = go.Figure()

fig.update_layout(
    height = 800,
    width = 800
)

fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=vals, name = 'sub Problem costs'))
fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=cost, name = 'scenario costs'))

for i in range(len(m)):
    fig.add_trace(go.Scatter(x=np.arange(rng[0],rng[1]), y=[c[i]-m[i]*x for x in np.arange(rng[0],rng[1])], opacity=0.2 ) )

fig.show()

In [12]:
vals

[25400335.400562387,
 23718798.316411406,
 22120348.073116705,
 20623672.423026036,
 19222011.94378155,
 17921257.796616126,
 16742733.550750112,
 15691592.562633883,
 14767714.361717308,
 13970649.146539578,
 13308493.486679645,
 12780311.503274461,
 12396662.55820656,
 12128941.959712775,
 11979926.099647934,
 11951035.469975406,
 12043989.12323813,
 12249521.542459458,
 12553895.014430283,
 12960563.758661699,
 13460547.345108902,
 14046020.101197403,
 14709414.241395876,
 15439113.260469284,
 16230446.828387579,
 17074297.12505545,
 17960216.694533877,
 18882974.523106687,
 19834448.283003222,
 20808231.542644173]

Modelling 2 stage SP for train timetable Allowance

let $\\$
$r_{i,i+1}$ be the ideal time required to traverse block section between stations $i$ and $i+1$ $\\$
$R_{i,i+1}$ be the Random Variable of traversal time for block $i$ - $i+1$ $\\$

$D_i$ be announced departure time of train at station $i$ $\\$
$a_i \ \& \  d_i$ be actual arrivals and departures at station $i$ $\\$

Model : 

<center>

$\min \displaystyle\sum_{\omega \in \Omega} p_\omega \left( \sum_i^n E_ie_i^\omega + L_il_i^\omega  \right)$

</center>

$\qquad \hspace{250pt} $ s.t. 

<center>

$D_i \geq D_{i-1} + r_{i-1,i} + h_i \hspace{20pt} \forall i \in S$

$\hspace{30pt} a_i^\omega = d_{i-1}^\omega + R_{i-1,i}^\omega \hspace{40pt} \forall i \in S, \omega \in \Omega$

$\hspace{30pt} a_i^\omega + h_i + e_i^\omega - l_i^\omega = D_i \hspace{20pt} \forall i \in S, \omega \in \Omega$

$\hspace{30pt} d_i^\omega = D_i + l_i^\omega \hspace{60pt} \forall i \in S, \omega \in \Omega$

$D_i,e_i^\omega,l_i^\omega \geq 0$

</center>